In [8]:
import requests
import re

In [36]:
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
"Accept-Encoding": "gzip, deflate", 
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8", 
"Dnt": "1", 
"Host": "httpbin.org", 
"Upgrade-Insecure-Requests": "1", 
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36", 
}
import random
user_agent_list = [
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
#Pick a random user agent
headers = {'User-Agent': random.choice(user_agent_list)}
#headers["User-Agent"] = random.choice(user_agent_list)
wiki = requests.get(
    'https://en.wikipedia.org/wiki/List_of_one-hit_wonders_in_the_United_States', headers=headers)
print(headers)
print(type(wiki), type(wiki.status_code))


{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
<class 'requests.models.Response'> <class 'int'>


In [25]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki.text, 'html.parser')


In [11]:
import pandas as pd


In [26]:
songhtml = soup.find_all('li')[14:580]
songs = [dict(zip(['Artist', 'Song'], [eachItem.getText()
              for eachItem in eachSong.find_all('a')[:2]])) for eachSong in songhtml]


In [13]:
headers = {'User-Agent': random.choice(user_agent_list)}
lyricSearch = requests.get(
    f"https://www.google.com/search?q={songs[35]['Artist']} {songs[35]['Song']} lyrics", headers=headers)

In [14]:
lyricSoup = BeautifulSoup(lyricSearch.text, 'html.parser')


In [15]:
# a = ''.join([eachPart.getText() for eachPart in lyricSoup.find_all("div", {"class": "BNeawe tAd8D AP7Wnd"})])


In [16]:
songtest = songs[:10].copy()

In [17]:
import time
counter = 1
for eachSong in songtest:
    headers = {'User-Agent': random.choice(user_agent_list)}
    lyricSearch = requests.get(
    f"https://www.google.com/search?q={eachSong['Artist']} {eachSong['Song']} lyrics", headers=headers)
    lyricSoup = BeautifulSoup(lyricSearch.text, 'html.parser')
    
    lyrics = lyricSoup.find("div", {"data-lyricid": re.compile(".*[Ll]yric.*")})
    if lyrics != None:
        eachSong['Lyrics'] = re.sub("(\w?[a-z?])([A-Z]\w?)","\g<1>\n\g<2>",lyrics.getText().split("Source")[0])
    # for i in range(3):
    #     #print(i, len(lyricSoup.find_all("div", {"class": "kCrYT"})[i].getText()))
    #     #print(lyricSoup.find_all("div", {"class": "kCrYT"})[i])
    #     #eachSong['Lyrics'] = ''.join([eachPart.getText() for eachPart in lyricSoup.find_all("div", {"class": "BNeawe tAd8D AP7Wnd"})][:-1])
    #     potentialLyrics = lyricSoup.find_all("div", {"class": "kCrYT"})[i].getText()
    #     if len(potentialLyrics) > 300:
    #         eachSong['Lyrics'] = potentialLyrics
    #         break
    #     elif i == 2:
    #         potentialLinks = lyricSoup.find("div", {"class": "kCrYT"}).find_all("a", href = True)
    #         foundLink = False
    #         for eachLink in potentialLinks:
    #             link = f"http{eachLink.get('href').split('http')[-1].split('&')[0]}"
    #             if re.match('.*youtube.*', link) == None:
    #                 eachSong['Lyrics'] = f'Lyrics not on Google, use : {link}'
    #                 headers = {'User-Agent': random.choice(user_agent_list)}
    #                 secondTry = requests.get(link, headers=headers)
    #                 moreSoup = BeautifulSoup(secondTry.text, 'html.parser')
    #                 eachSong['Lyrics'] += (f'\n{moreSoup.find("div", {"class": re.compile(".*[Ll]yric.*")})}')
    #                 print(moreSoup.prettify())
    #                 foundLink = True
    #                 break
    #             pass
    #         if foundLink == False:
    #             morePotentialLinks = lyricSoup.find_all("a", href = True)
    #             for additionalLinks in morePotentialLinks:
    #                 link = f"http{additionalLinks.get('href').split('http')[-1]}"
    #                 if (re.match('.*youtube.*', link) == None) and (re.match('.*search.*', link) == None) and (re.match('.*google.*', link) == None) and ((re.match('.*song.*', link) != None) or (re.match('.*lyric.*', link) != None)):
    #                     eachSong['Lyrics'] = f'Lyrics not on Google, use : {link}'
    #                     headers = {'User-Agent': random.choice(user_agent_list)}
    #                     secondTry = requests.get(link, headers=headers)
    #                     moreSoup = BeautifulSoup(secondTry.text, 'html.parser')
    #                     eachSong['Lyrics'] += (f'\n{moreSoup.find("div", {"class": re.compile(".*[Ll]yric.*")}).getText()}')
    #                     break
    #                 pass
    #             pass
    #         pass
    #     pass
    time.sleep(.5)
    counter += 1

In [18]:
songswlyrics = [eachSong for eachSong in songtest if 'Lyrics' in eachSong.keys()]

In [19]:
print(songswlyrics)

[{'Artist': 'The Crows', 'Song': 'Gee', 'Lyrics': "Do do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do do\nLove that girl\nOh Gee, my oh Gee, well o-o Gee\nWhy I love that girl, love that girl\nOh please, listen to me, hear-hear-hear my plea\nWhy I love that girl\nHold me, Baby, squeeze me\nNever let me go\nI'm not takin' chances\nBecause I love her, I love her so\nOh Gee, yes I love her, Yes I need her\nWhy I love that girl, love that girl\nHold me, Baby, squeeze me\nNever let me go\nI'm not takin' chances\nBecause I love her,  I love her so\nMy-my oh Gee, well o Gee, my-eye oh Gee\nWhy I love that girl, love that girl\nOh please, listen to me, hear-ear my plea\nWhy I love that girl, love that girl\nWhy I love that girl"}, {'Artist': 'Penguins', 'Song': 'Earth Angel', 'Lyrics': "Earth Angel, earth angel\nWill you be mine?\nMy darling dear, love you all the time\nI'm just a fool, A fool in love with you\nEarth Angel, ea

In [20]:
import json

In [21]:
with open("mydata.json", "w") as final:
   json.dump(songswlyrics, final, indent=2)

In [23]:
from datasets import Dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset = Dataset.from_list(my_list)